In [5]:
import pandas as pd
import re
import tabulate
import datetime as dt
import jupyter

Purpose: This task is to simulate a data cleaning request using a provided
German Ebay Car file.  Once the data has been cleaned I will perform some analysis,
such as determining average car price, the distribution of ads by date, and representation
of car brands.

In [7]:
def adjust_headers(autos):
    columns = autos.columns
    titles = []
    for title in columns:
        snake = re.sub('([A-Z]+)', r'_\1', title).lower()
        titles.append(snake)
    autos.columns = titles
    # There was a specific request to rename 4 rows which will be done below
    autos.rename({'year_of_registration': 'registration_year'}, axis=1, inplace=True)
    autos.rename({'month_of_registration': 'registration_month'}, axis=1, inplace=True)
    autos.rename({'not_repaired_damage': 'unrepaired_damage'}, axis=1, inplace=True)
    autos.rename({'date_created': 'ad_created'}, axis=1, inplace=True)
    autos.rename({'nr_of_pictures':'num_of_pictures'}, axis=1, inplace=True)
    autos.rename({'kilometer': 'odometer_km'}, axis=1, inplace=True)

    return autos

The format of all column titles are Camel Case (ex: camelCase).  The request specified to replace all instances of Camel Case with Snake Case.  To efficiently do so, I have utilized regex to filter through the file's headers, recognize the Camel Case pattern and replace it with Snake Case.  The supplemental renaming below was due to the prompt's specification for such titles.

In [8]:
def german_to_english(autos):


    seller_dict = {'privat':'private', 'gewerblich':'commercial'}
    sell = autos['seller']
    autos['seller'] = sell.map(seller_dict)

    offer_dict = {'Angebot':'Offer', 'Gesuch': 'Request'}
    of = autos['offer_type']
    autos['offer_type'] = of.map(offer_dict)

    vehicle_dict = {'coupe': 'coupe', 'kleinwagen': 'compact', 'limousine': 'limousine',
                    'cabrio': 'convertible', 'bus': 'bus', 'kombi': 'station wagon',
                    'andere': 'other'}
    vehicle_type = autos['vehicle_type']
    autos['vehicle_type'] = vehicle_type.map(vehicle_dict)

    gear_dict = {'manuell': 'manual', 'automatik': 'automatic'}
    gear = autos['gearbox']
    autos['gearbox'] = gear.map(gear_dict)

    fuel_dict = {'benzin': 'petrol', 'diesel': 'diesel', 'lpg': 'lpg', 'andere': 'alternative',
                 'hibrid': 'hybrid', 'cng': 'cng', 'elektro': 'electric'}
    fuel = autos['fuel_type']
    autos['fuel_type'] = fuel.map(fuel_dict)

    repair_dict = {'ja': 'yes', 'nein': 'no'}
    repair = autos['unrepaired_damage']
    autos['unrepaired_damage'] = repair.map(repair_dict)

    return autos

In order to make the results more readable for non-english speakers I have created a function to replace all instances with german words with their english counterparts.

In [9]:
def determine_descriptions(autos):
    columns = autos.columns
    '''
    for entry in columns:
        formatted = autos[entry].describe
        print(formatted)
    '''
   # print(autos['num_of_pictures'].describe)
    autos.drop('num_of_pictures', axis=1, inplace=True)
    return autos

After running the above function, we can see that all of the values for the "Number of Photos" column are all 0.  Therefore, we will remove it from the dataset.

In [10]:
def explore_price_od(autos):
   # print(autos[['odometer_km', 'price']].describe())
    '''
    Price IQR = 7200-1150 = 6050
    Price LFence = 1150 - (1.5*6050) = -7,925
    Price UFence = 7200 + (1.5*6050) = 16,275
    While it is ok that our lower fence is negative, we will go ahead and remove all
    zero values from the price column as it is not relatevent to our search

    odometer IQR = 150000 - 125000 = 25,000
    O LFence = 125000 - (1.5*25000) = 87,500
    O UFence = 150000 + (1.5*25000) = 187,500
    We can also see that our upper fence will never be reached as the highest value is
    150000.
    '''
    price_upper_fence = 7200 + (1.5*6050)
    autos = autos[autos['price'].between(1, price_upper_fence)]

    odom_lower_fence = 125000-(1.5*25000)
    autos = autos[autos['odometer_km'].between(odom_lower_fence, 150000)]
  #  print(autos[['odometer_km','price']].describe())

In [11]:
def explore_date(autos):
  #  print(autos['date_crawled'].str[:10].unique().size)  # counts the unique crawl dates

    date_crawled = (autos['date_crawled']
                    .str[:10]
                    .value_counts(normalize=True, dropna=False)
                    .round(3)
                    .sort_index()
                    )
 #   print(tabulate.tabulate(date_crawled.to_frame(), headers=['Date', '% crawled'], tablefmt='psql'))

In [12]:
def main():
    autos = pd.read_csv('autos.csv', encoding='Latin-1')
    autos = adjust_headers(autos)
    autos = german_to_english(autos)
   # determine_descriptions(autos)
    explore_price_od(autos)
    explore_date(autos)

In [13]:
if __name__ == '__main__':
    main()

<bound method NDFrame.describe of 0         0
1         0
2         0
3         0
4         0
         ..
371523    0
371524    0
371525    0
371526    0
371527    0
Name: num_of_pictures, Length: 371528, dtype: int64>
34
+------------+-------------+
| Date       |   % crawled |
|------------+-------------|
| 2016-03-05 |       0.026 |
| 2016-03-06 |       0.014 |
| 2016-03-07 |       0.036 |
| 2016-03-08 |       0.033 |
| 2016-03-09 |       0.034 |
| 2016-03-10 |       0.033 |
| 2016-03-11 |       0.033 |
| 2016-03-12 |       0.036 |
| 2016-03-13 |       0.016 |
| 2016-03-14 |       0.036 |
| 2016-03-15 |       0.033 |
| 2016-03-16 |       0.03  |
| 2016-03-17 |       0.032 |
| 2016-03-18 |       0.013 |
| 2016-03-19 |       0.035 |
| 2016-03-20 |       0.036 |
| 2016-03-21 |       0.036 |
| 2016-03-22 |       0.032 |
| 2016-03-23 |       0.032 |
| 2016-03-24 |       0.03  |
| 2016-03-25 |       0.033 |
| 2016-03-26 |       0.032 |
| 2016-03-27 |       0.03  |
| 2016-03-28 |       0.0